In [4]:
a, b, c = [1, 2, 3]
print(a)

1


In [5]:
import numpy as np
from skimage.filters import threshold_otsu
from skimage.measure import label
from skimage.io import imsave, imread
import napari
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from scipy import ndimage
import roifile

%gui qt

def blob(pos, center, size, intensity=10, xy_symmetric=True):
    """3D Gaussian
    pos: 3 arrays of x, y, and z values
    center : tuple of 3 floats for the gaussian center position
    size: tuple of 3 floats for the gaussian widths

    """
    x, y, z = pos
    # The idea is to make the computation in a 5D array
    # of shape ()
    
    x = x[np.newaxis, :, :, :] # 1, nx, ny, nz
    y = y[np.newaxis, :, :, :]
    z = z[np.newaxis, :, :, :]
    
    x0, y0, z0 = center.T
    x0 = x0[:, np.newaxis, np.newaxis, np.newaxis] # n_cells, 1, 1, 1
    y0 = y0[:, np.newaxis, np.newaxis, np.newaxis]
    z0 = z0[:, np.newaxis, np.newaxis, np.newaxis]
    sx, sy, sz = size.T[:, :, np.newaxis, np.newaxis, np.newaxis]
    
    intensity = intensity[:, np.newaxis, np.newaxis, np.newaxis]# n_cells, 1, 1, 1
    
    if xy_symmetric:
        sy = sx
        

    print(((x - x0)/sx).shape)
    # broadcasting
    sig = (
        np.exp(-(((x - x0) / sx) ** 2))
        * np.exp(-(((y - y0) / sy) ** 2))
        * np.exp(-(((z - z0) / sz) ** 2))
        * intensity
    ) # n_cells, n_x, n_y, n_z
    print(sig.shape)
    sig = sig.sum(axis=0) # n_x, n_y, n_z
    return sig


def generate_blobs(num_cells, shape, dtype, seed=0):
    """ Creates an image of shape `shape` with num_cells blobs in it
    """
    rng = np.random.default_rng(seed)
    cell_size = (5, 5, 1)
    cell_size_std = (1, 1, 0.01)
    max_int = 100
    max_int_std = 30

    centers = rng.integers(low=np.zeros(3), high=shape, size=(num_cells, 3))

    sizes = rng.normal(loc=cell_size, scale=cell_size_std, size=(num_cells, 3))

    intensities0 = rng.normal(loc=max_int, scale=max_int_std, size=num_cells)
    intensities1 = rng.normal(loc=max_int, scale=max_int_std, size=num_cells)

    
    stack0 = np.zeros(shape)
    stack1 = np.zeros(shape)
    
    
    pos = np.mgrid[ :shape[0], :shape[1], :shape[2]] # xyz
    
    stack0 += blob(pos, centers, sizes, intensities0)
    stack1 += blob(pos, centers, sizes, intensities1)
    #print(f"z = {z} / {shape[-1]}")
    
    stack0 = (stack0 / stack0.max()) * dtype(-1) # np.uint8 -> 255
    stack1 = (stack1 / stack1.max()) * dtype(-1)
    
    stack0 = stack0.clip(0).swapaxes(0, 2).astype(dtype)  # zyx
    stack1 = stack1.clip(0).swapaxes(0, 2).astype(dtype)  # zyx
    
    return stack0, stack1


def make_3canals(num_cells, shape, dtype, seed=0):
    """ Creates a 3 canals stack with the blobs, a transformed value
    (as a way to add diversity) and the labels from a simple thresholding
    """
    canal0, canal1 = generate_blobs(num_cells, shape, dtype, seed=seed)
    # apply threshold
    thresh = threshold_otsu(canal0)
    bw = canal0 > thresh
    canal2 = label(bw).astype(dtype) # max 255 cells !!
    
    return np.stack([canal0, canal1, canal2]) 



def create_tiff(fname, num_cells, shape, dtype=np.uint8):
    """Creates an image with make_3canals and saves it to fname
    num_cells: the number of blobs in the image
    shape : tuple of ints, the image shape (as x, y, z)
    dtype : the image dtype (default uint8)
    """
    image = make_3canals(num_cells, shape, dtype)
    imsave(fname, image, plugin="tifffile", metadata={'axes': 'CZYX'})
    return image

def main():
    shape = (1024, 1024, 10)
    num_cells = 1000
    image = create_tiff("test.tif", num_cells, shape)
    return image

In [6]:
max_lbl = 0
[0, 1, 2, 3]

max_lbl = 3
[3, 4, 5, 6]



[3, 4, 5, 6]

In [ ]:

def chunked(shape, n_cells, chunk_size=100, dtype=np.uint16):
    cell_lots =  n_cells // chunk_size

    stack = np.zeros(
        (3, shape[-1], shape[0], shape[1]), #CZXY
        dtype=np.uint8
    ) 
    max_lbl = 0
    # parallelize ?
    for i in range(cell_lots):
        print(stack.shape)
        print(f"\t## cell lot {i} / {cell_lots}")
        sstack = make_3canals(chunk_size, shape, dtype, seed=i)
        sstack[2, :, :][sstack[-1]!=0] += max_lbl # boolean indexing
        sstack[:2, ...] = sstack[:2, ...] // cell_lots
        max_lbl =a
        
        
        stack += sstack

    stack[-1, ...] -= stack[-1, ...].min()
    return stack
    
shape = (1024, 1024, 4) # XYZ
n_cells = 1000
stack = chunked(shape, n_cells)
print(stack.shape)


(3, 4, 1024, 1024)
	## cell lot 0 / 10
(100, 1024, 1024, 4)


In [8]:
napari.view_image(stack, channel_axis=0)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 511.5, 511.5), zoom=0.3807429130009775, angles=(0.0, 0.0, 90.0), interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 0.0, 1.0), (0.0, 1023.0, 1.0), (0.0, 1023.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x2e8b26ba4c0>, <Image layer 'Image [1]' at 0x2e8b26bab80>, <Image layer 'Image [2]' at 0x2e8b5e646a0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right'), active_layer=<Image layer 'Image [2]' at 0x2e8b5e646a0>, help='', status='Ready', theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000002E8A9C8EF70>], _persisted_mouse_event={}, _m

In [6]:
imsave("nap2.tif", stack, plugin="tifffile", metadata={'axes': 'CZYX'})
